In [1]:
# libraries

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

import pandas as pd
import re
from deep_translator import GoogleTranslator
import tqdm
import os

In [2]:
# helper functions:
def myTranslate(t):
    # Translate to English:
    text = GoogleTranslator(source='auto', target='en').translate(t)
    return text
    
def extractNumber(t):
    phone = re.findall(r'[[0-9][0-9 .\-\(\)]{6,}[0-9]', t)
    return phone

def getLabelData(elem):
    ts = elem.text.split("\n")
    name = ts[0]
    
    # TODO 
#     name = myTranslate(name)
    
    reviews = ts[1]
    try:
        address = ts[2]
    except:
        address = ""
        
#     address = myTranslate(address)

#     temp = name + " || " + address
#     trans = myTranslate(temp)
#     try:
#         name,address = trans.split("||")
#     except:
#         print("Issue with splitting ::",trans)
#         name = myTranslate(name)
#         address = myTranslate(address)
    
    
    # try replace
    address = address.replace("Pharmaceutical company ·","")
    
    try:
        timings_phone = ts[3]
    except:
        timings_phone = ""
        
    temp = extractNumber(timings_phone)
    if len(temp)>0:
        phonenumber = temp[0]
    else:
        phonenumber = ""
    # adding website
    
    try:
        a = elem.find_element(By.TAG_NAME,"a")
        website = a.get_attribute('href')
    except:
        website = ""
        
    obj = {
        "name":name.strip(),
        "phone":phonenumber,
        "website":website,
        "address":address.strip()
        
    }
    return obj


In [4]:
# Initiating the browser
options = webdriver.ChromeOptions()
path = "../chrome\chromedriver.exe"
service = Service(executable_path=path)
browser = webdriver.Chrome(service=service,options=options)
browser.maximize_window()

### 1. High level search:

In [3]:
country = "Egypt" # Please change accordingly

In [5]:
company_category = "Pharmaceutical Companies"
url = "https://www.google.com/maps/search/pharma companies in {}".format(country)
url

'https://www.google.com/maps/search/pharma companies in Egypt'

In [6]:
data = []

In [7]:
# started extracting
print("Currently extracting from :: ",url)

# Open the Google Map URL
browser.get(url)
# scrolling : 
actionChains = ActionChains(browser)

# scrolling upto 20 times::
end = False
repeat = 0
limit = 28
    
while not end and repeat < limit:

    # check end
    end_identify = browser.find_elements("xpath",'.//span[@class="HlvSq"]')
    if len(end_identify)>0:
        end = True
        print("Reached end !!")

    repeat += 1
    if repeat == limit:
        print("28 times reached !!")
        end = True
    
    elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
    if len(elems)==0:
        print("Remember to add ")
        continue
    elem = elems[-1]
    
    xoffset = 0
    yoffset = 80 # If you get "target move out of boundaries" => change this value
    
    actionChains.move_to_element_with_offset(elem, xoffset, yoffset).perform()
    browser.execute_script("arguments[0].scrollIntoView();", elem)




Currently extracting from ::  https://www.google.com/maps/search/pharma companies in Egypt
28 times reached !!


In [8]:
# extracting the data
elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
for elem in elems:
    d = getLabelData(elem)
    data.append(d)


C:\Users\yeshu\AppData\Local\Temp\ipykernel_10980\595260018.py:8: FutureWarning: Possible nested set at position 1
  phone = re.findall(r'[[0-9][0-9 .\-\(\)]{6,}[0-9]', t)


In [9]:
## saving results:
df = pd.DataFrame(data)
df.head()

,name,phone,website,address
0,Egyptian Company For Pharmaceutical,20 2 22250000,http://www.eptc-egydrug.com/,"1353 Corniche El Nil, Shobra"
1,Merck Ltd. /Merck Biopharma/Merck life science...,20 2 22686614,http://merckgroup.com/,Merck Ltd. - Egypt | Building 4 Section 1150 E...
2,BIO PHARMA EGYPT s.a.e,20 2 38204351,,"5 Mossadak St Dokki Giza Governorate EG, 12311"
3,Chemipharm Pharmaceutical Industries,20 2 33474875,,6 Hassan Sabry St
4,2S Pharma Group,20 2 24472341,http://www.2spharma.com/,2110 Al Arqam Ibn Aby Al Arqam St. Off Ring Rd...


In [10]:
# creating a folder
os.mkdir("../results/names/{}".format(country))

In [11]:
df.to_csv("../results/names/{}/companies_1.csv".format(country),index=None)

#### Cites wise

In [12]:
# loading the list of cities
cities_df = pd.read_csv("../data/worldcities.csv")
cities = list(cities_df[cities_df['country'] == country]['city_ascii'].values)

In [13]:
len(cities)

159

In [14]:
cities

['Cairo',
 'Giza',
 'Alexandria',
 'Shubra al Khaymah',
 'Al Mansurah',
 'Halwan',
 'Al Mahallah al Kubra',
 'Port Said',
 'Suez',
 'Al `Ashir min Ramadan',
 'Tanta',
 'Asyut',
 'Biba',
 "Ash Shuhada'",
 'Az Zaqaziq',
 'Al Fayyum',
 'Al `Ajami',
 'Al Matariyah',
 'Ismailia',
 'Kafr ad Dawwar',
 'Aswan',
 'Damanhur',
 'Al Minya',
 'Mallawi',
 'Damietta',
 'Qina',
 'Bani Suwayf',
 'Timayy al Imdid',
 'Luxor',
 'Suhaj',
 'Shibin al Kawm',
 'Al `Arish',
 'Banha',
 'Al Ghardaqah',
 'Kafr ash Shaykh',
 'Madinat as Sadis min Uktubar',
 'Disuq',
 'Bilbays',
 'Idfu',
 'Mit Ghamr',
 'Sinnuris',
 'Rosetta',
 'Jirja',
 'Munuf',
 'Faqus',
 'Qalyub',
 'Matruh',
 'Idku',
 'Akhmim',
 'Zifta',
 'Al Badrashayn',
 'Talkha',
 'Manfalut',
 'Al Khankah',
 'Bani Mazar',
 'Kirdasah',
 'Kafr az Zayyat',
 'Maghaghah',
 '`Izbat al Burj',
 'Al Qusiyah',
 'Abnub',
 'Tukh',
 'Hihya',
 'Al Minshah',
 'Al Qurayn',
 'Dalja',
 'As Saff',
 'Samannud',
 'Abu Qir',
 'Al Karnak',
 'Al Balyana',
 'Al Kharjah',
 'Al `Ayyat',

In [15]:
## Remember to drag , zoom in and then extract : repeat
## Initiating the browser
options = webdriver.ChromeOptions()
path = "../chrome\chromedriver.exe"
service = Service(executable_path=path)
browser = webdriver.Chrome(service=service,options=options)
browser.maximize_window()

# browser.set_page_load_timeout(300) # only for 5 minutes

In [16]:
for city in tqdm.tqdm(cities[57:]):
    data = []
    url = "https://www.google.com/maps/search/Pharmaceutical companies in {} {}".format(city,country)
#     print("Currently extracting from :: ",url)
    browser.get(url)
    
    # scrolling : 
    actionChains = ActionChains(browser)

    # scrolling upto 20 times::
    end = False
    repeat = 0
    limit = 28

    while not end and repeat < limit:

        # check end
        end_identify = browser.find_elements("xpath",'.//span[@class="HlvSq"]')
        if len(end_identify)>0:
            end = True
#             print("Reached end !!")

        repeat += 1
        if repeat == limit:
#             print("28 times reached !!")
            end = True

        elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
        if len(elems)==0:
#             print("Remember to add ")
            continue
        elem = elems[-1]

        xoffset = 0
        yoffset = 80

        actionChains.move_to_element_with_offset(elem, xoffset, yoffset).perform()
        browser.execute_script("arguments[0].scrollIntoView();", elem)
        time.sleep(0.2)


    # extracting the data
    elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
    for elem in elems:
        d = getLabelData(elem)
        data.append(d)

#     print("Done")
#     print("Extracted :: ",len(elems))
    
#     print("Saving the file:")
    
    df = pd.DataFrame(data)
    df.to_csv("../results/names/{}/{}_companies.csv".format(country,city),index=None)
    

100%|█████████████████████████████████████████████████████████████████████████████| 102/102 [5:02:21<00:00, 177.86s/it]


In [ ]:
# Note: There are times when you get Aw, Swap error from google maps: then do below
# Check the number of iteration done till now 
# continue iterating cities from that city
# Run again

#### Selected City - Do Zoom in | Drag Drop | Extract || Repeat

In [23]:
data = []

In [24]:
city = "Riyadh" # Mostly do it for capital city
url = "https://www.google.com/maps/search/Pharmaceutical companies in {} {}".format(city,country)
#     print("Currently extracting from :: ",url)
browser.get(url)

In [25]:
## zoom in 
zoom_button = browser.find_element("xpath",'.//button[@id="widget-zoom-in"]')
action = ActionChains(browser)
action.click(on_element = zoom_button)
action.perform() 

In [26]:
## search this area
search_this_area = browser.find_element("xpath",'.//span[@class="Kw7Rxf"]')
action.click(on_element = search_this_area)
action.perform() 

In [28]:
## Extracting again
# scrolling : 
actionChains = ActionChains(browser)

# scrolling upto 20 times::
end = False
repeat = 0
limit = 28
    
while not end and repeat < limit:

    # check end
    end_identify = browser.find_elements("xpath",'.//span[@class="HlvSq"]')
    if len(end_identify)>0:
        end = True
        print("Reached end !!")

    repeat += 1
    if repeat == limit:
        print("28 times reached !!")
        end = True
    
    elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
    if len(elems)==0:
        print("Remember to add ")
        continue
    elem = elems[-1]
    
    xoffset = 0
    yoffset = 80
    
    actionChains.move_to_element_with_offset(elem, xoffset, yoffset).perform()
    browser.execute_script("arguments[0].scrollIntoView();", elem)
    time.sleep(0.2)


# extracting the data
elems = browser.find_elements("xpath",'.//div[@class="bfdHYd Ppzolf OFBs3e  "]')
for elem in elems:
    d = getLabelData(elem)
    data.append(d)
    
print("Done")
print("Extracted :: ",len(elems))

Reached end !!
Done
Extracted ::  5


In [29]:
# filtering duplicates:
## removing duplicates
final_data = []
for d in data:
    if d not in final_data:
        final_data.append(d)

In [30]:
len(final_data)

21

In [31]:
df = pd.DataFrame(final_data)

In [32]:
df.to_csv("../results/names/{}/Companies_Rest_All.csv".format(country,city),index=None)